# LAMMPS Tutorials 01 nvt. Running your first LAMMPS simulation!

### Author: ABDELLAH TAHIRI 

Please contact me if you have a problem with this tutorial, so I can modify in Github.  I have added FAQs, and will update my versions of LAMMPS in the future to keep the scripts current.

The latest version of this [Jupyter Notebook](http://ipython.org/notebook.html) tutorial is available at https://github.com/athiri78/DM/edit/main/LAMMPS-Tutorials.



***

## Description:
<a id="Sec1"></a>

This script performs a molecular dynamics simulation of a system of atoms interacting via the Lennard-Jones potential. The system is first equilibrated in an NVE ensemble (constant energy), and then it is further equilibrated under the NVT ensemble (constant temperature) with a Nose-Hoover thermostat. The simulation outputs atomic positions throughout the run, and the system's temperature and energy are tracked.
The script is useful for studying the properties of simple atomic systems, such as the equilibrium dynamics of atoms in a Lennard-Jones fluid, with the ability to observe the effects of temperature control and energy conservation..

<img src="https://icme.hpc.msstate.edu/mediawiki/images/e/ef/box.gif" alt="nvt" title="box" />

## Step 1: Download an Input File
<a id="Step1"></a>

This input script was run using the Aug 2024 version of LAMMPS. Changes in some commands may require revision of the input script. To get the input file, you have a few options:

*  Copy the text below and paste it into a text file, `nvt.in`. Use the `Paste Special` command with unformatted text into the file. 
*  Or, I added the command `%%writefile nvt.in` to the Jupyter Notebook which should just do everything for you!




In [26]:
%%writefile nvt.in
######################################
# ------------------------- Initialize Simulation ------------------------------
units lj                  # Use Lennard-Jones derived units
atom_style atomic         # Use atomic model (each atom is a point particle)

# ------------------------- Neighbor and Atom Setup --------------------------
neighbor 1 bin            # Bin-based neighbor list with a skin of 1.0
atom_modify map array sort 10 6  # Store atom data in an array and sort every 10 steps, max neighbors = 6

# ------------------------- Define Simulation Box ----------------------------
boundary p p p            # Periodic boundary conditions in x, y, z directions
region bbox block 0 10 0 10 0 10   # Define a cubic simulation box from 0 to 10 in all directions

# ------------------------- Create Box and Atoms ----------------------------
create_box 1 bbox         # Create simulation box for 1 atom type
lattice fcc 0.25          # Define a FCC lattice with a density of 0.25 in LJ units
create_atoms 1 box        # Populate the box with atoms of type 1 (argon)
mass 1 1.0                # Set the mass of atom type 1 to 1.0 (LJ units)

# ------------------------- Initial Configuration ---------------------------
dump dcheck all xyz 1 initial.xyz   # Dump atom positions every timestep into 'initial.xyz'
run 0                       # Run 0 timesteps to output initial configuration

undump dcheck               # Remove the initial dump after initial configuration

# ------------------------- Set Up Force Field ------------------------------
pair_style lj/cut 2.5       # Use Lennard-Jones potential with a cutoff of 2.5
pair_coeff * * 1.0 1.0      # Set Lennard-Jones parameters: epsilon = 1.0, sigma = 1.0

# ------------------------- Initialize Velocities ---------------------------
velocity all create 1.0 5493108 rot yes dist gaussian  # Create velocities with Gaussian distribution at T = 1.0 (LJ units)

timestep 0.001             # Set the simulation timestep to 0.001 LJ time units

# ------------------------- NVE Ensemble (Energy Conservation) --------------
dump dnve all xyz 1 nve.xyz   # Output atom positions to 'nve.xyz' every timestep
fix nve_int all nve          # Apply NVE ensemble (constant N, V, E)
thermo 10                    # Output thermodynamic data every 10 timesteps

run 1000                     # Run the simulation for 1000 timesteps

undump dnve                  # Remove dump for NVE simulation
unfix nve_int                # Remove the NVE fix after the run

# ------------------------- NVT Ensemble (Temperature Control) --------------
dump dnvt all xyz 1 nvt.xyz   # Output atom positions to 'nvt.xyz' every timestep
fix nvt_nh all nvt temp 1.0 1.0 0.5  # Apply NVT ensemble with Nose-Hoover thermostat at T = 1.0, damping time = 0.5
fix com_remove all momentum 100 linear 1 1 1  # Remove center-of-mass motion

run 10000                    # Run the simulation for 10,000 timesteps

print "All done!"

Overwriting nvt.in


Awesome!  That little script should have written the above text to the file `nvt.in`.  To check, let's execute a command on the present directory listing all files that end in `*.in`.

In [28]:
!dir *.in

 Le volume dans le lecteur C n’a pas de nom.
 Le numéro de série du volume est DC02-C1A6

 Répertoire de C:\Users\pc\Desktop\M2A

21/11/2024  10:17             1 774 calc_fcc.in
12/11/2024  14:40             1 565 calc_fcc_ver1.in
09/11/2024  14:10             1 233 calc_fcc_ver2.in
12/11/2024  16:49             1 177 lj.in
12/11/2024  17:08               533 ljbarosta.in
12/11/2024  17:19             1 389 ljovito.in
12/11/2024  18:10             1 018 ljrdf.in
12/11/2024  16:59               522 ljthermo.in
23/11/2024  15:26             3 010 nvt.in
               9 fichier(s)           12 221 octets
               0 Rép(s)  162 065 788 928 octets libres


***
## Step 2: Running LAMMPS
### Run this using LAMMPS in Jupyter Notebook
We can actually run this from Jupyter Notebook.  Let's try it.

In [30]:
!lmp -in nvt.in  -pk omp 8 -sf omp 

LAMMPS (29 Aug 2024 - Update 1)
OMP_NUM_THREADS environment is not set. Defaulting to 1 thread. (src/comm.cpp:98)
  using 1 OpenMP thread(s) per MPI task
Loaded 1 plugins from C:\LAMMPS 64-bit 29Aug2024\plugins
set 8 OpenMP thread(s) per MPI task
using multi-threaded neighbor list subroutines
Created orthogonal box = (0 0 0) to (10 10 10)
  1 by 1 by 1 MPI processor grid
Lattice spacing in x,y,z = 2.5198421 2.5198421 2.5198421
Created 256 atoms
  using lattice units in orthogonal box = (0 0 0) to (10 10 10)
  create_atoms CPU = 0.001 seconds
No /omp style for force computation currently active
Setting up Verlet run ...
  Unit style    : lj
  Current step  : 0
  Time step     : 0.005
Per MPI rank memory allocation (min/avg/max) = 5.81 | 5.81 | 5.81 Mbytes
   Step          Temp          E_pair         E_mol          TotEng         Press     
         0   0              0              0              0              0            
Loop time of 6e-06 on 8 procs for 0 steps with 256 atoms

0.0

Sweet!

If you want to view the entire file (opening `log.lammps` in Notepad), then:

In [31]:
!type log.lammps

LAMMPS (29 Aug 2024 - Update 1)
OMP_NUM_THREADS environment is not set. Defaulting to 1 thread. (src/comm.cpp:98)
  using 1 OpenMP thread(s) per MPI task
Loaded 1 plugins from C:\LAMMPS 64-bit 29Aug2024\plugins
package omp 8
set 8 OpenMP thread(s) per MPI task
using multi-threaded neighbor list subroutines
######################################
# ------------------------- Initialize Simulation ------------------------------
units lj                  # Use Lennard-Jones derived units
atom_style atomic         # Use atomic model (each atom is a point particle)

# ------------------------- Neighbor and Atom Setup --------------------------
neighbor 1 bin            # Bin-based neighbor list with a skin of 1.0
atom_modify map array sort 10 6  # Store atom data in an array and sort every 10 steps, max neighbors = 6

# ------------------------- Define Simulation Box ----------------------------
boundary p p p            # Periodic boundary conditions in x, y, z directions
region bbox block 0

***
## LAMMPS Input Script Explained

In this section, we will break down what LAMMPS is doing for each step. The easy way to do this on your own is to consult the LAMMPS manual for each command or go to the Internet LAMMPS manual, *i.e.*, at https://lammps.sandia.gov

***

## Simulation Units and Atom Style
<a id="Step1"></a>
* Units: lj (Lennard-Jones units):
* The simulation uses Lennard-Jones (LJ) derived units, where energy, length, and mass are normalized to unit values based on the LJ potential. This means that:
* Energy is measured in units of the depth of the potential well (ε),
* Length is measured in units of the distance at which the potential is zero (σ),
* Mass is normalized accordingly.
## Atom Style: atomic:
This style represents atoms as point particles, each having only a position, velocity, and mass, with interatomic forces modeled by potentials.
## Neighboring List: neighbor 1 bin:
Sets the neighbor list with a binning algorithm for better efficiency in finding neighboring atoms within a certain cutoff distance. The 1 indicates the skin distance beyond the cutoff.
## Atom Modify: atom_modify map array sort 10 6:
This command tells LAMMPS to store atomic data in an array format and to sort atoms every 10 steps, considering the maximum number of neighbors as 6. This is important for efficient memory and force calculations.
## Simulation Box Setup
* Boundary Conditions: boundary p p p:
This command sets periodic boundary conditions in all three directions (x, y, z), meaning atoms exiting one side of the box re-enter from the opposite side.
* Region: region bbox block 0 10 0 10 0 10:
This defines the simulation box (bbox) as a cubic region from 0 to 10 in all directions (x, y, z).
* Create Box: create_box 1 bbox:
A simulation box is created for a single atom type (argon or other) within the defined region bbox.
 ##  Atom Creation
* Lattice: lattice fcc 0.25:
The system uses a face-centered cubic (FCC) lattice with a density defined by the parameter 0.25 in LJ units.
* Create Atoms: create_atoms 1 box:
This command populates the simulation box with atoms of type 1, following the defined FCC lattice structure.
* Mass: mass 1 1.0:
Sets the mass of atom type 1 to 1.0 in LJ units.
## Dump Command: dump dcheck all xyz 1 initial.xyz:
This dumps the positions of all atoms into an initial.xyz file every timestep (1 timestep in this case) for visualization purposes.
## Description of the LAMMPS Input Script
This LAMMPS input script defines a simulation that uses the Lennard-Jones (LJ) potential and explores the dynamics of a simple atomic system using three different ensembles: NVE (constant number of atoms, volume, and energy), NVT (constant number of atoms, volume, and temperature), and Lennard-Jones interactions. Below is a detailed breakdown of the script's key components and their functions:
## Simulation Units and Atom Style
Units: lj (Lennard-Jones units):
The simulation uses Lennard-Jones (LJ) derived units, where energy, length, and mass are normalized to unit values based on the LJ potential. This means that:
Energy is measured in units of the depth of the potential well (ε),
Length is measured in units of the distance at which the potential is zero (σ),
Mass is normalized accordingly.
Atom Style: atomic:
This style represents atoms as point particles, each having only a position, velocity, and mass, with interatomic forces modeled by potentials.
##  Neighboring List Setup
Neighboring List: neighbor 1 bin:
Sets the neighbor list with a binning algorithm for better efficiency in finding neighboring atoms within a certain cutoff distance. The 1 indicates the skin distance beyond the cutoff.
Atom Modify: atom_modify map array sort 10 6:
This command tells LAMMPS to store atomic data in an array format and to sort atoms every 10 steps, considering the maximum number of neighbors as 6. This is important for efficient memory and force calculations.
## Simulation Box Setup
Boundary Conditions: boundary p p p:
This command sets periodic boundary conditions in all three directions (x, y, z), meaning atoms exiting one side of the box re-enter from the opposite side.
Region: region bbox block 0 10 0 10 0 10:
This defines the simulation box (bbox) as a cubic region from 0 to 10 in all directions (x, y, z).
Create Box: create_box 1 bbox:
A simulation box is created for a single atom type (argon or other) within the defined region bbox.
## Atom Creation
Lattice: lattice fcc 0.25:
The system uses a face-centered cubic (FCC) lattice with a density defined by the parameter 0.25 in LJ units.
Create Atoms: create_atoms 1 box:
This command populates the simulation box with atoms of type 1, following the defined FCC lattice structure.
Mass: mass 1 1.0:
Sets the mass of atom type 1 to 1.0 in LJ units.
Dump Command: dump dcheck all xyz 1 initial.xyz:
This dumps the positions of all atoms into an initial.xyz file every timestep (1 timestep in this case) for visualization purposes.
## Run 0:
A run of 0 timesteps is executed just to dump the initial configuration before any dynamics are applied.
## Undump:
After dumping the initial configuration, the dump command is removed (undump dcheck).
## Force Field Setup
Pair Style: pair_style lj/cut 2.5:
The interaction between the atoms is described by the Lennard-Jones potential with a cutoff distance of 2.5 LJ units. This defines how atoms interact based on their distance, with a cutoff to limit calculations to nearby atoms.
Pair Coefficients: pair_coeff * * 1.0 1.0:
Sets the Lennard-Jones potential parameters (ε = 1.0 and σ = 1.0) for all atom pairs. These values govern the strength and range of the interactions.
## Velocity: velocity all create 1.0 5493108 rot yes dist gaussian:
Initializes the velocities of all atoms at a temperature of 1.0 in LJ units. The velocities are assigned randomly with a Gaussian distribution, and the "rot" flag ensures the system starts with zero angular momentum.
## Timestep: timestep 0.001:
The simulation timestep is set to 0.001 LJ units of time. This defines the time interval between each step in the simulation.
## NVE Ensemble (Microcanonical Ensemble)
* Dump Command: dump dnve all xyz 1 nve.xyz:
The atomic positions are output to a file (nve.xyz) every timestep, allowing for tracking of atom positions during the NVE simulation.
* Fix NVE: fix nve_int all nve:
Applies the NVE ensemble (constant number of atoms, volume, and energy), where the system is allowed to evolve with the total energy conserved.
* Thermo Output: thermo 10:
Outputs thermodynamic information (such as energy and temperature) every 10 timesteps.
* Run Command: run 1000:
Runs the simulation for 1000 timesteps in the NVE ensemble, simulating the system's behavior without temperature control (energy conservation only).
* Undump & Unfix:
Removes the dump and fix commands once the NVE run is complete.
## NVT Ensemble (Canonical Ensemble) with Nose-Hoover Thermostat
* Dump Command: dump dnvt all xyz 1 nvt.xyz:
The atomic positions are again output every timestep, this time during the NVT simulation.
* Fix NVT: fix nvt_nh all nvt temp 1.0 1.0 0.5:
This applies the NVT ensemble with a Nose-Hoover thermostat to control the temperature of the system. The system is maintained at a constant temperature of 1.0 LJ units, with a relaxation time of 0.5 LJ units of time.
* Fix Momentum: fix com_remove all momentum 100 linear 1 1 1:
This command removes any center-of-mass motion from the system, ensuring the system remains centered at the origin.
* Run Command: run 10000:
Runs the simulation for 10,000 timesteps in the NVT ensemble, allowing the system to equilibrate at the desired temperature.

***
## FAQs 
<br>
<div class="alert alert-danger">
<strong>Question 1</strong>: How do the results from the NVE ensemble differ from the NVT ensemble in terms of system energy and temperature stability
</div>

This question evaluates how the total energy and temperature behave under different ensembles. The NVE ensemble (microcanonical) conserves energy, whereas the NVT ensemble (canonical) controls temperature with a Nose-Hoover thermostat.

<br>
<div class="alert alert-danger">
<strong>Question 2</strong>: What is the effect of the initial velocity distribution and the thermostat damping time on system equilibration and temperature control?
</div>

This question aims to explore how the initial velocities (randomly created at 1.0 LJ temperature) and the thermostat damping time (0.5 in the NVT ensemble) impact the equilibration process and the stabilization of temperature
<br>
<div class="alert alert-danger">
<strong>Question 3</strong>: How do the system's structural properties (e.g., atomic positions and lattice configuration) evolve over time under different ensembles?.
</div>

* Use the dumped XYZ files (initial.xyz, nve.xyz, and nvt.xyz) to visualize the atomic positions at different timesteps, and compare the lattice structure. Look for any distortion in the lattice in NVE versus NVT, and observe whether the FCC lattice is maintained over time.
* You can also calculate structural properties, such as the Radial Distribution Function (RDF) or Bond Order Parameter, to compare structural changes.



***
## Tutorial Links

[Click here to open the next tutorial](LAMMPS-Tutorials-02.ipynb)